In [ ]:
import pandas as pd
import numpy as np
import csv 
from google.colab import drive
import os
import re
# drive.mount('/content/drive')
# workdir = '/content/drive/MyDrive/FactGrid Cuneiform (AWCA) (1)/geography' # for Tina
# workdir = '/content/drive/MyDrive/FactGrid Cuneiform (AWCA)/geography' # for Adam colab
workdir = '/Users/aa/ANE_PDF_SHARE/ocr-output/' # for Adam to run locally
# workdir = '/Users/aa/ANE_PDF_SHARE/ocr-output/' # for Adam local MBP

Mounted at /content/drive


## Make a catalog of the folders and page.csv files

We can use this notebook to make the initial catalog with the directory paths:
https://colab.research.google.com/drive/1v118lH5R9sd31dAWEZUD60rRe929rkRA?usp=sharing

1. We first run the index_directory.py
2. That file generates an index.txt [(which we uploaded here)](https://drive.google.com/file/d/1Ctrc3iNnNBchYmPXT5nfT2bf0zkieFEF/view?usp=share_link)
3. We need to convert the index.txt to a catalog.csv file, which will remove all the stuff except for the paths to each page.csv for each document
4. We can then run this notebook locally to make a words.csv for each document

In [ ]:
# read the CSV file into a DataFrame
# occurrence_list = pd.read_csv("/content/drive/MyDrive/FactGrid Cuneiform (AWCA)/geography/index.txt", sep = '\t',header=None) # when running from Colab
occurrence_list = pd.read_csv("/Users/aa/ANE_PDF_SHARE/ocr-output/index.txt", sep = '\t',header=None) # when Adam runs locally

# # convert all columns to string type to avoid mixed types issues
# occurrence_list = occurrence_list.astype(str)
# occurrence_list = occurrence_list.fillna('')

# # concatenate the contents of the 8 columns into a new column called 'catalog'
# occurrence_list['catalog'] = occurrence_list[[0, 1, 2, 3, 4, 5, 6, 7]].apply(lambda x: ''.join(x), axis=1)
# occurrence_list['catalog'] = occurrence_list['catalog'].str.rstrip('nan')

# # drop the original 8 columns
# occurrence_list.drop(columns=[0, 1, 2, 3, 4, 5, 6, 7, 8], inplace=True)

# filter the rows that contain 'page.csv' in the catalog column
occurrence_list = occurrence_list.loc[occurrence_list[0].str.contains('page.csv')]

occurrence_list


,0
2,/Users/aa/ANE_PDF_SHARE/ocr-output/67052_Hasan...
5,/Users/aa/ANE_PDF_SHARE/ocr-output/111822_noor...
7,/Users/aa/ANE_PDF_SHARE/ocr-output/72709_Cambr...
10,/Users/aa/ANE_PDF_SHARE/ocr-output/62033_tlir/...
13,/Users/aa/ANE_PDF_SHARE/ocr-output/36297_black...
...,...
422195,/Users/aa/ANE_PDF_SHARE/ocr-output/124280_Mach...
422198,/Users/aa/ANE_PDF_SHARE/ocr-output/92562_ZZ Wi...
422201,/Users/aa/ANE_PDF_SHARE/ocr-output/61394_kronn...
422204,/Users/aa/ANE_PDF_SHARE/ocr-output/83770_Benne...


In [ ]:
print(occurrence_list.iloc[23,:])

0    /Users/aa/ANE_PDF_SHARE/ocr-output/54928_Vleem...
Name: 63, dtype: object


In [ ]:
occurrence_list.to_csv('index_3.csv')

In [ ]:
print(occurrence_list.iloc[1,0])

/Users/aa/ANE_PDF_SHARE/ocr-output/1_Akanuma-IronKaman(2002)/page.csv


In [ ]:
print('/Users/aa/ANE_PDF_SHARE/ocr-output/67052_Hasanlu/page.csv')

/Users/aa/ANE_PDF_SHARE/ocr-output/67052_Hasanlu/page.csv


In [ ]:
bookname = occurrence_list.iloc[1,0].split('/')[-2]
book_index = re.search(r'(\d+)_', os.path.basename(bookname))
book_index

<re.Match object; span=(0, 2), match='1_'>

This is the final loop to save word.csv files to each corresponeded ocr-ouput file

In [ ]:
for i in range(len(occurrence_list)):
  bookname = occurrence_list.iloc[i,0].split('/')[-2]
  book_index = re.search(r'(\d+)_', os.path.basename(bookname))
  if book_index:
      book_index = book_index.group(1)
  else:
      book_index = None
  try:
    csv_path = occurrence_list.iloc[i,0]
    # Read the CSV file
    pages = pd.read_csv(csv_path, index_col=0)
  except FileNotFoundError:
    try:
      csv_path = occurrence_list.iloc[i,0]
      pages = pd.read_csv(csv_path, index_col=0)
    except FileNotFoundError:
      print(occurrence_list.iloc[i,0])
  # Create an empty list to store the new DataFrame rows
  new_rows = []

  # Iterate through the rows in the 'pages' DataFrame
  for index, row in pages.iterrows():
      # Split the text into words
      if pd.notna(row['text']):
        # Change the text format to string
        row['text'] = str(row['text'])
        # Split the text into words
        words = row['text'].split()
      
      # Create a new row for each word and add it to the 'new_rows' list
      for i, word in enumerate(words):
          identifier = f"{book_index}.{index}.{i}"
          new_row = {'identifier': identifier, 'word': word}
          new_rows.append(new_row)
  words_df = pd.DataFrame()
  # Create a new DataFrame from the 'new_rows' list
  words_df = pd.DataFrame(new_rows)
  try: 
    save_path = f'{workdir}/{bookname}/words.csv'
  except OSError:
    save_path = f'{workdir}/{bookname} /words.csv'
  # print(words_df)
  # print(save_path)
  words_df.to_csv(save_path)

/Users/aa/ANE_PDF_SHARE/ocr-output/0_Attinger - A propos de AK «faire» (I) (ZA 952005) 46-64/page.csv


NameError: ignored

In [ ]:
# for i in range(3):
#   bookname = ocurrance_list.iloc[i,0]
#   book_index = re.search(r'(\d+)_', os.path.basename(bookname))
#   if book_index:
#       book_index = book_index.group(1)
#   else:
#       book_index = None
#   try:
#     csv_path = f'{workdir}/{bookname}/{ocurrance_list.iloc[i,1]}'
#     # Read the CSV file
#     pages = pd.read_csv(csv_path, index_col=0)
#   except FileNotFoundError:
#     try:
#       csv_path = f'{workdir}/{bookname} /{ocurrance_list.iloc[i,1]}'
#       pages = pd.read_csv(csv_path, index_col=0)
#     except FileNotFoundError:
#       print(f"{bookname}/{ocurrance_list.iloc[i,1]}")
#   # Create an empty list to store the new DataFrame rows
#   new_rows = []

#   # Iterate through the rows in the 'pages' DataFrame
#   for index, row in pages.iterrows():
#       # Split the text into words
#       if pd.notna(row['text']):
#         # Split the text into words
#         words = row['text'].split()
      
#       # Create a new row for each word and add it to the 'new_rows' list
#       for i, word in enumerate(words):
#           identifier = f"{book_index}.{index}.{i}"
#           new_row = {'identifier': identifier, 'word': word}
#           new_rows.append(new_row)
#   words_df = pd.DataFrame()
#   # Create a new DataFrame from the 'new_rows' list
#   words_df = pd.DataFrame(new_rows)
#   save_path = f'{workdir}/{bookname}/words.csv'
#   # print(words_df)
#   # print(save_path)
#   words_df.to_csv(save_path)

      identifier          word
0          0.0.0            46
1          0.0.1        Pascal
2          0.0.2      Attinger
3          0.0.3             A
4          0.0.4        propos
...          ...           ...
11237   0.18.707  la-ba-an-ak-
11238   0.18.708             e
11239   0.18.709             .
11240   0.18.710             à
11241   0.18.711        suivre

[11242 rows x 2 columns]
/content/drive/MyDrive/FactGrid Cuneiform (AWCA) (1)/geography/ocr-output/0_Attinger - A propos de AK «faire» (I) (ZA 95, 2005) 46-64/words.csv
         identifier              word
0         10000.0.0               Der
1         10000.0.1          Aufstieg
2         10000.0.2               des
3         10000.0.3  neubabylonischen
4         10000.0.4           Reiches
...             ...               ...
13260  10000.20.335              1965
13261  10000.20.336                R.
13262  10000.20.337                D.
13263  10000.20.338          Barnett,
13264  10000.20.339            Keeper

[

In [ ]:
words_df

,identifier,word
0,0.0.0,46
1,0.0.1,Pascal
2,0.0.2,Attinger
3,0.0.3,A
4,0.0.4,propos
...,...,...
11237,0.18.707,la-ba-an-ak-
11238,0.18.708,e
11239,0.18.709,.
11240,0.18.710,à


## Next steps:

__Identify language__
1. concat the words.csv files
2. download all languages (lexemes + forms) from wikidata
3. match Wikidata forms to words.csv (left join)

__Other issues:__
1. OCR error:  äpfel OCR epfel 
  a. edit distance
  b. cosine similarity
2. proper nouns (names of people and places)
  a. we can make a list to curate for NER
3. inproper OCR (e.g. Arabic) = no matches at all...
  a. highlight for re-ocr with proper langauge